In [1]:
# setting oneapi env and installing deps
# !./p_custom2.sh

# !python -m pip install mkl

!source /opt/intel/oneapi/setvars.sh --force

!pip install intel-extension-for-pytorch --no-cache-dir
!pip install torch --no-cache-dir
!pip install peft

 
:: initializing oneAPI environment ...
   bash: BASH_VERSION = 5.1.16(1)-release
   args: Using "$@" for setvars.sh arguments: --force
:: advisor -- latest
:: ccl -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: embree -- latest
:: inspector -- latest
:: intelpython -- latest

CommandNotFoundError: Your shell has not been properly configured to use 'conda deactivate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



CommandNotFoundError: Your shell has not been properly configured to use 'conda deactivate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  -

In [ ]:
import numpy as np
import pandas as pd
import subprocess
import time
import intel_extension_for_pytorch as ipex

from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer
import torch

/home/uc4ddc6536e59d9d8f8f5069efdb4e25/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mh_dir='mh_one_api'
with_intel_optimization=True
without_intel_optimization=True

pred_file_name="full_pred9.csv"
pred_file_path=f"/home/uc4ddc6536e59d9d8f8f5069efdb4e25/{mh_dir}/data/custom_pred/{pred_file_name}"
start_index=subprocess.check_output("tail "+pred_file_path+" -n 1 | awk -F' ' '{print $1}'",shell=True)
print(start_index)
start_index,end_index=0,5

f_test_path=f"/home/uc4ddc6536e59d9d8f8f5069efdb4e25/{mh_dir}/data/f_testd.csv"
f_test_data=pd.read_csv(f_test_path)


checkpoint_dir=f"/home/uc4ddc6536e59d9d8f8f5069efdb4e25/{mh_dir}/model/ft_models/flan-t5-xl_peft_ft_v2/"
checkpoint_name=subprocess.check_output(f"ls {checkpoint_dir} | grep checkpoint | tail -1",shell=True)
checkpoint_name=str(checkpoint_name).replace("b'","").replace("\\n'","")
checkpoint_path=checkpoint_dir+checkpoint_name
model_path=checkpoint_path



NameError: name 'subprocess' is not defined

In [7]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
model = AutoPeftModelForSeq2SeqLM.from_pretrained(model_path, )

Loading checkpoint shards: 100%|██████████| 2/2 [00:34<00:00, 17.11s/it]


In [44]:

if with_intel_optimization==True:
    global imodel
    # imodel = model.to('xpu')
    imodel = ipex.optimize(model)

RuntimeError: PyTorch is not linked with support for xpu devices

In [42]:
# print(subprocess.check_output(". /opt/intel/oneapi/setvars.sh ",shell=True))
# print(subprocess.check_output("sycl-ls",shell=True))
# print(subprocess.check_output("./sys_info.sh"))

print("----------------------running inference on finetuned model---------------")
print("-----------------metrics with vs without intel oneapi optimization------------")
print("---inference time without intel oneapi optimization: %s seconds ---" % (pt))
print("---inference time with intel oneapi optimization: %s seconds ---" % (pti))
print(f"---inference time reduced (%) by using intel one api optimization: {(pt-pti)*100/pt} % ---")


----------------------running inference on finetuned model---------------
-----------------metrics with vs without intel oneapi optimization------------
---inference time without intel oneapi optimization: 113.74230933189392 seconds ---
---inference time with intel oneapi optimization: 99.55642342567444 seconds ---
---inference time reduced (%) by using intel one api optimization: 12.471951720995776 % ---


In [43]:

# ------predict--------------
bs=1
end_index=len(f_test_data)
for i in range(start_index,end_index,bs):
    print(f"predicting {i} to {i+bs-1} prompt")
    prompts = f_test_data.loc[i:i+bs-1,['input','instruction']].values.tolist()
    prompts=prompts
    t_prompts=[]
    for p in prompts:
        context=str(p[0])
        # context=str(p[0])
        question=p[1]
        t_prompts.append([context,question])
        # t_prompts+=[f"input: {context}\n\ninstruction: {question}"]
    prompts=t_prompts
        
    # print(prompts)
    res=[]
    input_ids = tokenizer(prompts, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
    # sample up to 30 tokens
    torch.manual_seed(0)  # doctest: +IGNORE_RESULT

    if without_intel_optimization==True:
        start_time = time.time()
        outputs = model.generate(input_ids=input_ids, do_sample=True, max_length=150)
        pt=time.time() - start_time
        print(f"--------------time taken by normal model={pt} seconds")


    if with_intel_optimization==True:
        # input_ids=input_ids.to('xpu')
        start_time = time.time()
        outputs = imodel.generate(input_ids=input_ids, do_sample=True, max_length=150)
        pti=time.time() - start_time
        print(f"--------------time taken by optimized model={pti} seconds")

        if without_intel_optimization==True:
            # print("---------------system info---------------------------")
            # print(subprocess.check_output("source /opt/intel/oneapi/setvars.sh && sycl-ls",shell=True))
            print("-----------------metrics with vs without intel oneapi optimization------------")
            print("---inference time without intel oneapi optimization: %s seconds ---" % (pt))
            print("---inference time with intel oneapi optimization: %s seconds ---" % (pti))
            print(f"---inference time reduced (%) by using intel one api optimization: {(pt-pti)*100/pt} % ---")


    res+=tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
    # Writing data to a file
    with open(pred_file_path, "a+") as file1:
        file1.writelines(f"{i+i1} $$ {res[i1]}\n" for i1 in range(len(res)))
    print(f"\n-------------wrote {i} to {i+bs-1} preds")

print("-----------------Prediction_finished-----------------------")
# print(subprocess.check_output("scancel $((SLURM_JOB_ID+1))",shell=True))

predicting 0 to 0 prompt


--------------time taken by normal model=113.90422749519348 seconds
--------------time taken by optimized model=100.90329360961914 seconds
-----------------metrics with vs without intel oneapi optimization------------
---inference time without intel oneapi optimization: 113.90422749519348 seconds ---
---inference time with intel oneapi optimization: 100.90329360961914 seconds ---
---inference time reduced (%) by using intel one api optimization: 11.413916912015361 % ---

-------------wrote 0 to 0 preds
predicting 1 to 1 prompt
--------------time taken by normal model=116.72430205345154 seconds
--------------time taken by optimized model=137.21207332611084 seconds
-----------------metrics with vs without intel oneapi optimization------------
---inference time without intel oneapi optimization: 116.72430205345154 seconds ---
---inference time with intel oneapi optimization: 137.21207332611084 seconds ---
---inference time reduced (%) by using intel one api optimization: -17.5522756720167

KeyboardInterrupt: 